In [2]:
import time
import faiss
import numpy as np

### faiss チュートリアル

In [59]:
def f(s, diff, dim=10):
    ans = [s]
    for i in range(dim-1):
        s += diff
        ans.append(s)
    return ans

In [76]:
d = 10                           # dimension
nb = 6                   # database size
nq = 1                       # nb of queries
xb = np.asarray([f(0,1), f(0,2), f(10, 1), f(100, 1), f(100,2), f(1000,1)])
print("データ: \n", xb, "\n")
xb = xb.astype('float32')

データ: 
 [[   0    1    2    3    4    5    6    7    8    9]
 [   0    2    4    6    8   10   12   14   16   18]
 [  10   11   12   13   14   15   16   17   18   19]
 [ 100  101  102  103  104  105  106  107  108  109]
 [ 100  102  104  106  108  110  112  114  116  118]
 [1000 1001 1002 1003 1004 1005 1006 1007 1008 1009]] 



In [77]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
6


In [78]:
np.set_printoptions(suppress=True)
k = 6
xq = np.asarray([xb[5] + 0.1]).astype('float32')
print("クエリ: \n", xq, "\n")
D, I = index.search(xq, k)     # actual search
print("距離: \n", np.round(D,3))
print("近傍点のインデックス: \n", I)

クエリ: 
 [[1000.1 1001.1 1002.1 1003.1 1004.1 1005.1 1006.1 1007.1 1008.1 1009.1]] 

距離: 
 [[       0.1  8021076.   8101799.   9802979.   9912276.  10001999. ]]
近傍点のインデックス: 
 [[5 4 3 2 1 0]]


### faiss 計算時間調査

In [14]:
d = 128                # 128次元ベクトル
nb = 100000         # データ数 10万
nq = 1                   # クエリ数 1 
np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')  # データ
xq = np.random.random((nq, d)).astype('float32')  #クエリ
print("データの次元: {}".format(xb.shape) )
print("クエリの次元: {}".format(xq.shape))

データの次元: (100000, 128)
クエリの次元: (1, 128)


In [15]:
index = faiss.IndexFlatL2(d)  # インデックス作成
print("インデックス生成済み: {}".format(index.is_trained))
index.add(xb)                          # インデックスにデータを追加
print("インデックス生成済みデータ数: {}".format(index.ntotal))

インデックス生成済み: True
インデックス生成済みデータ数: 100000


In [16]:
np.set_printoptions(suppress=True)
k = index.ntotal                     # 検索する近傍点の数 10万

start = time.time()
D, I = index.search(xq, k)     # 検索
end = time.time()

t = end - start
h = t//3600
m = (t-(3600*h))//60
s = t-(3600*h)-(60*m)

print("距離ベクトルの次元: {}".format(D.shape))
print("近傍点のインデックスの次元: {}".format(I.shape))
print("データ数: {0}, クエリ数: {1}, 検索時間: {2}時間{3}分{4}秒".format(nb, nq, h, m, s))

距離ベクトルの次元: (1, 100000)
近傍点のインデックスの次元: (1, 100000)
データ数: 100000, クエリ数: 1, 検索時間: 0.0時間0.0分0.1002507209777832秒
